In [1]:
import pandas as pd
import numpy as np
import time
import csv
import tensorflow as tf
import matplotlib.pyplot as plt
from mpl_finance import candlestick2_ochl,volume_overlay
import talib
from math import log, exp
from matplotlib import dates as mdates
from matplotlib import ticker as mticker
from matplotlib.dates import DateFormatter, WeekdayLocator, DayLocator, MONDAY,YEARLY
from matplotlib.dates import MonthLocator,MONTHLY
import datetime as dt
import pylab
import h5py
from PIL import Image
import os
import threading
import multiprocessing as mp
from multiprocessing import Pool
import random
from numba import vectorize, float64, cuda
import imageio
%matplotlib inline


from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation,BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50

Using TensorFlow backend.


In [2]:
stocknum = {'0051','1102','1216','1227','1314','1319','1434','1451','1476','1477','1504','1536','1560','1590',
            '1605','1704','1717','1718','1722','1723','1789','1802','1909','2015','2049','2059','2106','2201',
            '2204','2207','2227','2231','2312','2313','2324','2327','2337','2344','2347','2352','2353','2356',
            '2360','2371','2376','2377','2379','2385','2439','2448','2449','2451','2478','2492','2498','2542',
            '2603','2606','2610','2615','2618','2723','2809','2812','2834','2845','2867','2888','2912','2915',
            '3019','3034','3044','3051','3189','3231','3406','3443','3532','3673','3682','3702','3706','4137',
            '4915','4943','4958','5264','5522','5871','6005','6116','6176','6239','6269','6285','6409','6414',
            '6415','6452','6456','8454','8464','9910','9914','9917','9921','9933','9938','9941','9945'}

In [3]:
hdf5_path = "pic_sum_new.h5"

def generate_train_from_file(x,y,batch_size):
    
    ylen = len(y)
    loopcount = ylen // batch_size
    
    while(True):
        X_output = np.zeros((batch_size,) + (380, 383, 3), dtype=np.float64)
        i = randint(0,loopcount)
        for i,pic_addr in enumerate(x[i * batch_size : (i + 1) * batch_size]) :
            pic = Image.open(pic_addr)
            X_output[i] = pic
            
        yield X_output, y[i * batch_size : (i + 1) * batch_size]

        
def generate_val_from_file(path):
    while 1:
        f = h5py.File(path,'r')

        for pic_path,label in zip(f['val_img'],f['val_labels']):
                 
            pic = Image.open(pic_path)
            
            batch_input = np.zeros((1,) + (380, 383, 3), dtype=np.float64)
            batch_target = np.zeros((1,) + (4,), dtype=np.float64)

            batch_input[0] = pic
            batch_target[0] = label
        

            yield (batch_input, batch_target)
        f.close()

def main():

    model = ResNet50(input_shape=(380,383,3),classes=4,pooling='max',include_top=True,weights=None)
  
    model.summary()
    print ('\n\n\n\n')

    #filepath="best_acc.h5"
    #checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    # early_stop = EarlyStopping(monitor='acc', min_delta=0.0001, patience=5, verbose=1, mode='max')
 
    # callbacks_list = [checkpoint, early_stop]
    # myAdam = Adam(decay=0.1) 

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # train_history = model.fit(x=x_train, y=y_train, epochs=3, batch_size=100, shuffle=True, callbacks = callbacks_list)
    f = h5py.File(hdf5_path,'r')
    
    batch_size = 10
    
    train_history = model.fit_generator(generate_train_from_file(f['train_img'],f['train_labels'],batch_size), 
                                        steps_per_epoch=100,epochs=10
                                        ,verbose=1,workers=multiprocessing.cpu_count(), use_multiprocessing=True)
    
    loss, accuracy = model.evaluate_generator(sgenerate_train_from_file(f['train_img'],f['train_labels'],batch_size),
                                        workers=multiprocessing.cpu_count(), use_multiprocessing=True)
    
    print('\ntest loss: ', loss)
    print('\ntest accuracy: ', accuracy)
    
    #accPrint = int(accuracy * 10000)
    #model.save(str(accPrint) + '.h5')

main()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 380, 383, 3)  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 190, 192, 64) 9472        input_1[0][0]                    
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 190, 192, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 190, 192, 64) 0           bn_conv1[0][0]                   
__________________________________________________________________________________________________
max_poolin

bn3d_branch2a (BatchNormalizati (None, 47, 48, 128)  512         res3d_branch2a[0][0]             
__________________________________________________________________________________________________
activation_20 (Activation)      (None, 47, 48, 128)  0           bn3d_branch2a[0][0]              
__________________________________________________________________________________________________
res3d_branch2b (Conv2D)         (None, 47, 48, 128)  147584      activation_20[0][0]              
__________________________________________________________________________________________________
bn3d_branch2b (BatchNormalizati (None, 47, 48, 128)  512         res3d_branch2b[0][0]             
__________________________________________________________________________________________________
activation_21 (Activation)      (None, 47, 48, 128)  0           bn3d_branch2b[0][0]              
__________________________________________________________________________________________________
res3d_bran

NameError: name 'multiprocessing' is not defined